In [ ]:
import spimage
import h5py
import numpy as np
import scipy.ndimage as ndimage
import scipy.ndimage.morphology as morphology
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [ ]:
filename = "/Users/benedikt/data/LCLS/amol3416/pbcv/phasing/r0098/r0098_004.cxi"

In [ ]:
pixelsize  = 4*75*1e-6
distance   = 0.37
wavelength = 1.59e-9
shape = (256,256)
x_to_q = lambda x: 2*np.sin(np.arctan2(x*pixelsize, distance)/2.) / wavelength
dx = wavelength * distance / (shape[1] * pixelsize) * 1e9 #[nm/px]

In [ ]:
threshold = 0.04
with h5py.File(filename, 'r') as f:
    real_error = f['scores_final']['real_error'][:]
    selection  = real_error < threshold

In [ ]:
plt.figure()
plt.plot(np.sort(real_error))
plt.axhline(threshold, color='r')
plt.show()

In [ ]:
with h5py.File(filename, 'r') as f:
    fourier_space_final = f['fourier_space_final'][selection,:,:]
    real_space_final = f['real_space_final'][selection,:,:]
    support_final    = f['support_final'][selection,:,:]

In [ ]:
plt.figure()
plt.imshow(np.abs(fourier_space_final[0]), interpolation='none', cmap='magma', norm=colors.LogNorm())
plt.colorbar()
plt.show()

In [ ]:
output_prtf = spimage.prtf(real_space_final, support_final, translate=True, enantio=True)

In [ ]:
# Resulst from PRTF calculation
super_image = output_prtf['super_image']/selection.sum()
prtf_2d     = output_prtf['prtf']
fourier_super = np.fft.fftshift(np.fft.fftn(super_image))
th = np.mean(np.abs(super_image[~support_final[0]]))
st = np.std(np.abs(super_image[~support_final[0]]))
super_support = (np.abs(super_image) > th + 8*st)[c+1:-c,c+1:-c]

# Radial average of PRTF
nx, ny = prtf_2d.shape[1], prtf_2d.shape[0]
xx,yy = np.meshgrid(np.arange(nx),np.arange(ny))
mask_radial = np.sqrt((xx-nx/2)**2 + (yy-ny/2)**2) < nx/2
prtf_centers, prtf_radial = spimage.radialMeanImage(prtf_2d, output_r=True)
prtf_qr = x_to_q(prtf_centers)*1e-9
limit_qr = prtf_qr[np.abs(ndimage.gaussian_filter1d(prtf_radial,2) - (1/np.e)).argmin()]

In [ ]:
xx,yy = np.meshgrid(np.arange(super_support.shape[0]), np.arange(super_support.shape[0]))
boundary = (morphology.distance_transform_edt(super_support) == 1)
coords = np.transpose(np.vstack([xx[boundary], yy[boundary]]))
f,ij = find_furthest(coords)
support_largest_nm = f * dx
x1,x2 = coords[ij[0]], coords[ij[1]]

In [ ]:
c = 110
fig = plt.figure(figsize=(11,4))
axl = fig.add_subplot(121)
axl.set_title('PRTF (using %d images)' %selection.sum())
axl.plot(prtf_qr, prtf_radial, color='k')
axl.axhline(1/np.e, color='k', ls=':')
axl.text(prtf_qr[-1], 1/np.e, '1/e', va='bottom', ha='right')
#ax.axhline(1./np.sqrt(selection.sum()), color='0.75', ls='--')
axl.axvline(limit_qr, color='r', ls='--')
axl.text(limit_qr, .95, r'$\Delta$ = %.2f nm  ' %(1/limit_qr), va='top', ha='right', color='r')
axl.axvline(x_to_q(256./2)*1e-9, color='0.75', ls='-')
axl.text(x_to_q(260./2)*1e-9, .95, 'detector edge', va='top', ha='left', color='0.75')
axl.set_xlabel(r'Spatial frequency [nm$^{-1}$]')
axl.set_ylabel('PRTF')
axr  = fig.add_subplot(122)
axr.set_title('averaged (%d images)' %selection.sum())
axr.set_xticks([])
axr.set_yticks([])
axr.plot([x1[0],x2[0]], [x1[1],x2[1]], 'ro-', color='w')
axr.text(x2[0], x2[1]-1,'%d nm' %(support_largest_nm), va='bottom', ha='left', color='w')
im = axr.imshow(np.abs(super_image)[c+1:-c,c+1:-c], cmap='viridis', interpolation='none')
axr.add_patch(plt.Rectangle((2,32),100./7.7,1.2, facecolor='w', lw=0))
axr.text(8.3,31.5, r'100 nm', color='w', weight='bold', fontsize=10, ha='center', va='bottom')
cb = fig.colorbar(im)
cb.ax.set_ylabel('Electron density [arb. units]')
plt.show()

In [ ]:
fig = plt.figure(figsize=(11,4))
axl = fig.add_subplot(121)
im1 = axl.imshow(np.abs(fourier_super), 
           interpolation='none', cmap='magma', norm=colors.LogNorm(vmin=0.1))
axl.set_xticks([])
axl.set_yticks([])
fig.colorbar(im1)
axr = fig.add_subplot(122)
imr = axr.imshow(np.angle(fourier_super), 
           interpolation='none', cmap='Blues')
axr.set_xticks([])
axr.set_yticks([])
fig.colorbar(imr)
plt.show()